In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz

In [2]:
import os
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [3]:
!pip install findspark

In [4]:
import findspark
findspark.init()

In [5]:
!pip3 install pyspark==3.0.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.7/204.7 MB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 16.7 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.0.0-py2.py3-none-any.whl size=205044159 sha256=a91d560c6dd5ece447ba2abe997862d21ab769a4ba31c29f259dbbc157e7ee7d
  Stored in directory: /root/.cache/pip/wheels/b1/bb/8b/ca24d3f756f2ed967225b0871898869db676eb5846df5adc56
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7


In [6]:
# Инициализация

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.window import Window
from pyspark.sql.types import DoubleType, IntegerType, ArrayType, StringType
from pyspark.sql.functions import udf, explode, rank, col, max, sum, desc, countDistinct
import re
from typing import List
import pyspark.sql as sql

spark = SparkSession \
    .builder \
    .appName("Lab2_6407_Bykova") \
    .config("spark.jars.packages", "com.databricks:spark-xml_2.12:0.13.0") \
    .getOrCreate()

In [8]:
# Загрузка данных

import os
prog_path = '/content/programming-languages.csv'
posts_path = '/content/posts_sample.xml'

posts = spark.read.format('xml').options(rowTag='row').load(posts_path)


program = spark.read \
      .option("header", True) \
      .option("inferSchema", True) \
      .option("DateTimeFormat", 'M/d/y H:m') \
      .csv(prog_path)

**ЗАДАНИЕ**

Сформировать отчёт с информацией о 10 наиболее популярных языках программирования по итогам года за период с 2010 по 2020 годы. Отчёт будет отражать динамику изменения популярности языков программирования и представлять собой набор таблиц "топ-10" для каждого года.

Получившийся отчёт сохранить в формате Apache Parquet.

Для выполнения задания вы можете использовать любую комбинацию Spark API: RDD API, Dataset API, SQL API.

In [9]:
posts.printSchema()

root
 |-- _AcceptedAnswerId: long (nullable = true)
 |-- _AnswerCount: long (nullable = true)
 |-- _Body: string (nullable = true)
 |-- _ClosedDate: timestamp (nullable = true)
 |-- _CommentCount: long (nullable = true)
 |-- _CommunityOwnedDate: timestamp (nullable = true)
 |-- _CreationDate: timestamp (nullable = true)
 |-- _FavoriteCount: long (nullable = true)
 |-- _Id: long (nullable = true)
 |-- _LastActivityDate: timestamp (nullable = true)
 |-- _LastEditDate: timestamp (nullable = true)
 |-- _LastEditorDisplayName: string (nullable = true)
 |-- _LastEditorUserId: long (nullable = true)
 |-- _OwnerDisplayName: string (nullable = true)
 |-- _OwnerUserId: long (nullable = true)
 |-- _ParentId: long (nullable = true)
 |-- _PostTypeId: long (nullable = true)
 |-- _Score: long (nullable = true)
 |-- _Tags: string (nullable = true)
 |-- _Title: string (nullable = true)
 |-- _ViewCount: long (nullable = true)
 |-- _corrupt_record: string (nullable = true)



_Tags - информация о языках<br/>
_CreationDate - информация о дате<br/>
_ViewCount - популярность языков

In [10]:
posts = posts.select("_CreationDate", "_ViewCount", "_Tags")
posts.show(2, False)

+-----------------------+----------+------------------------------------------------------+
|_CreationDate          |_ViewCount|_Tags                                                 |
+-----------------------+----------+------------------------------------------------------+
|2008-07-31 21:42:52.667|42817     |<c#><floating-point><type-conversion><double><decimal>|
|2008-07-31 22:08:08.62 |18214     |<html><css><internet-explorer-7>                      |
+-----------------------+----------+------------------------------------------------------+
only showing top 2 rows



In [11]:
from pyspark.sql.functions import col

filtered_posts = posts.filter("_Tags is not null").rdd.map(lambda x: (str(x[0]).split('-')[0], x[1], x[2][1:-1].split("><"))).toDF(["year", "views", "languages"])
filtered_posts = filtered_posts.filter(col("year") >= 2010).filter(col("year") <= 2020)
filtered_posts.show(truncate=False)

+----+-----+---------------------------------------------------------------+
|year|views|languages                                                      |
+----+-----+---------------------------------------------------------------+
|2010|3650 |[c++, character-encoding]                                      |
|2010|617  |[sharepoint, infopath]                                         |
|2010|1315 |[iphone, app-store, in-app-purchase]                           |
|2010|973  |[symfony1, schema, doctrine, fixtures]                         |
|2010|132  |[java]                                                         |
|2010|419  |[visual-studio-2010, stylecop]                                 |
|2010|869  |[cakephp, file-upload, swfupload]                              |
|2010|1303 |[git, cygwin, putty]                                           |
|2010|748  |[drupal, drupal-6]                                             |
|2010|1258 |[php, wordpress, memory]                                       |

Столбец languages распакуем по примеру
<br/>Из записи: <br/>
|2010| 599| [c#, c++, java]|
<br/>Получилось 3 новых записи:<br/>
|2010| 599| c#  |<br/>
|2010| 599| c++ |<br/>
|2010| 599| java|<br/>

In [12]:
from pyspark.sql.functions import explode
exploded_posts = filtered_posts.select(filtered_posts.year, filtered_posts.views, explode(filtered_posts.languages)).withColumnRenamed("col", "language")
exploded_posts.show()

+----+-----+------------------+
|year|views|          language|
+----+-----+------------------+
|2010| 3650|               c++|
|2010| 3650|character-encoding|
|2010|  617|        sharepoint|
|2010|  617|          infopath|
|2010| 1315|            iphone|
|2010| 1315|         app-store|
|2010| 1315|   in-app-purchase|
|2010|  973|          symfony1|
|2010|  973|            schema|
|2010|  973|          doctrine|
|2010|  973|          fixtures|
|2010|  132|              java|
|2010|  419|visual-studio-2010|
|2010|  419|          stylecop|
|2010|  869|           cakephp|
|2010|  869|       file-upload|
|2010|  869|         swfupload|
|2010| 1303|               git|
|2010| 1303|            cygwin|
|2010| 1303|             putty|
+----+-----+------------------+
only showing top 20 rows



In [13]:
from pyspark.sql.functions import lower

prog = (program.drop("wikipedia_url")).withColumn("name",  lower("name"))
prog.show()

+------------+
|        name|
+------------+
|     a# .net|
|  a# (axiom)|
|  a-0 system|
|          a+|
|         a++|
|        abap|
|         abc|
|   abc algol|
|       abset|
|       absys|
|         acc|
|      accent|
|    ace dasl|
|        acl2|
|     act-iii|
|     action!|
|actionscript|
|         ada|
|     adenine|
|        agda|
+------------+
only showing top 20 rows



In [14]:
exploded_posts = exploded_posts.crossJoin(prog).where("language = name")
counted_posts = exploded_posts.groupBy("year", "language").agg({"views": "sum"})
counted_posts.show()

+----+-----------+----------+
|year|   language|sum(views)|
+----+-----------+----------+
|2013|     erlang|       864|
|2017| typescript|     18709|
|2013| javascript|    224453|
|2013|         f#|       228|
|2012| powershell|     13711|
|2019|        php|      1729|
|2017|    haskell|      2132|
|2013| autohotkey|       790|
|2019|      xpath|        27|
|2015|     racket|       762|
|2017|         go|       970|
|2018|     python|     35783|
|2017|     prolog|        76|
|2014|        php|    139737|
|2018|        x++|        78|
|2014|        lua|       604|
|2016|objective-c|       677|
|2014|      scala|     24440|
|2019|         go|       288|
|2018|        php|     14020|
+----+-----------+----------+
only showing top 20 rows



In [15]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

window = Window.partitionBy(counted_posts.year).orderBy(col("sum(views)").desc())
tempDF = counted_posts.withColumn("rank", row_number().over(window))
tempDF = tempDF.withColumnRenamed("sum(views)", "views")
tempDF.show()

+----+-----------+-----+----+
|year|   language|views|rank|
+----+-----------+-----+----+
|2016|     python|78849|   1|
|2016|       java|76563|   2|
|2016|        php|49559|   3|
|2016| javascript|43321|   4|
|2016| powershell|24230|   5|
|2016|          c|13092|   6|
|2016|       ruby|10533|   7|
|2016| typescript| 6158|   8|
|2016|          r| 5587|   9|
|2016|      scala| 4639|  10|
|2016|       bash| 3420|  11|
|2016|     matlab| 2097|  12|
|2016|         go| 1792|  13|
|2016|       curl| 1062|  14|
|2016|objective-c|  677|  15|
|2016|    labview|  569|  16|
|2016|       perl|  562|  17|
|2016| coldfusion|  538|  18|
|2016|    aspectj|  382|  19|
|2016|      alice|  262|  20|
+----+-----------+-----+----+
only showing top 20 rows



In [30]:
tempDF = tempDF.filter(col("rank") <= 10)
result = tempDF.drop("rank")
result = result.orderBy(col("year").asc(), col("views").desc())
result.show()

+----+-----------+------+
|year|   language| views|
+----+-----------+------+
|2010| javascript|232878|
|2010|       java|143138|
|2010|       ruby| 75287|
|2010|          c| 54927|
|2010|     matlab| 51865|
|2010|objective-c| 46232|
|2010|     python| 40736|
|2010|applescript| 31320|
|2010|        php| 25574|
|2010|          r|  7499|
|2011|     python|113640|
|2011| javascript| 89636|
|2011|       java| 77519|
|2011|          c| 62390|
|2011|        php| 47509|
|2011|objective-c| 29405|
|2011|       bash| 16091|
|2011|          r| 12764|
|2011|       ruby| 12572|
|2011|     cython|  8109|
+----+-----------+------+
only showing top 20 rows



In [53]:
directory_path = "result_data.parquet"
result.write.parquet(directory_path)

In [54]:
!ls

posts_sample.xml	   result_data.parquet	spark-3.1.1-bin-hadoop3.2
programming-languages.csv  sample_data		spark-3.1.1-bin-hadoop3.2.tgz


In [55]:
# Удаление

import shutil
shutil.rmtree(directory_path)